In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

import gc
import time

from utils import *

PATH = Path('data')
list(PATH.iterdir())

[PosixPath('data/submission'),
 PosixPath('data/train_day8_3to16_FE.feather'),
 PosixPath('data/sample_submission.csv'),
 PosixPath('data/train_day9_3to16_FE.feather'),
 PosixPath('data/xgb2.model'),
 PosixPath('data/train_sample.csv'),
 PosixPath('data/train_day7_3to16_FE.feather'),
 PosixPath('data/dtree.dot'),
 PosixPath('data/test_FE.feather'),
 PosixPath('data/xgb1.model'),
 PosixPath('data/val_idxs.p'),
 PosixPath('data/mean_enc_df'),
 PosixPath('data/xgb1_noratio.model'),
 PosixPath('data/validation')]

In [10]:
day_x = get_feather('test_FE.feather',PATH)
day_x.click_time = pd.to_datetime(day_x.click_time)

In [12]:
day_x.shape

(18790469, 16)

In [13]:
day_x.tail()

,ip,app,device,os,channel,click_time,channel_device_mean_target_day9_alpha1,app_os_mean_target_day9_alpha1,device_mean_target_day9_alpha1,os_device_mean_target_day9_alpha1,app_mean_target_day9_alpha1,ip_mean_target_day9_alpha1,app_device_mean_target_day9_alpha1,channel_mean_target_day9_alpha1,channel_os_mean_target_day9_alpha1,app_channel_mean_target_day9_alpha1
18790464,99442,9,1,13,127,2017-11-10 15:00:00,1.022593e-03,1.151977e-03,0.001664,0.001405,0.000941,0.002527,0.001020,0.001009,1.321021e-03,0.001009
18790465,88046,23,1,37,153,2017-11-10 15:00:00,1.440514e-04,2.137107e-07,0.001664,0.003325,0.000013,0.000757,0.000013,0.000140,4.455278e-05,0.000014
18790466,81398,18,1,17,265,2017-11-10 15:00:00,1.369547e-04,4.423326e-04,0.001664,0.001032,0.000473,0.000418,0.000477,0.000129,1.395637e-04,0.002527
18790467,123236,27,1,13,122,2017-11-10 15:00:00,4.268720e-04,5.716378e-04,0.001664,0.001405,0.002019,0.001958,0.002031,0.000412,5.404212e-04,0.000474
18790468,73516,12,2,27,265,2017-11-10 15:00:00,3.320501e-08,2.386684e-05,0.000287,0.000148,0.000097,0.001552,0.000006,0.000129,1.800671e-07,0.000098


# next click

In [ ]:
GROUP_BY_NEXT_CLICKS = [
    {'groupby': ['ip']},
    {'groupby': ['ip', 'app']},
    {'groupby': ['ip', 'channel']},
    {'groupby': ['ip', 'os']},
    {'groupby': ['ip', 'os', 'device']},
    {'groupby': ['ip', 'os', 'device', 'app']}
]
for spec in GROUP_BY_NEXT_CLICKS:
    next_fea = '{}_next_click'.format('_'.join(spec['groupby']))
    prev_fea = '{}_prev_click'.format('_'.join(spec['groupby']))
    # Run calculation
    print(f">> Grouping by {spec['groupby']} ...")
    day_x = next_prev_click(day_x,spec['groupby'],next_fea,prev_fea)

day_x= downcast_dtypes(day_x)

day_x.to_feather(PATH/'test_nextclick.feather')

# day_x = get_feather('test_nextclick.feather')

gc.collect()

In [14]:
# GROUP_BY_NEXT_CLICKS = [
#     {'groupby': ['ip']},
#     {'groupby': ['ip', 'app']},
#     {'groupby': ['ip', 'channel']},
# ]
# for spec in GROUP_BY_NEXT_CLICKS:
#     new_fea = '{}_next_click'.format('_'.join(spec['groupby']))
#     # Run calculation
#     print(f">> Grouping by {spec['groupby']}, and saving time to next click in: {new_fea}")
#     day_x = time_till_next_click(day_x,spec['groupby'],new_fea)

# day_x= downcast_dtypes(day_x)

# day_x.to_feather(PATH/'test_nextclick.feather')

# # day_x = get_feather('test_nextclick.feather')

# gc.collect()

>> Grouping by ['ip'], and saving time to next click in: ip_next_click
>> Grouping by ['ip', 'app'], and saving time to next click in: ip_app_next_click
>> Grouping by ['ip', 'channel'], and saving time to next click in: ip_channel_next_click


# Cumcount (for each day) (might be discard)

In [4]:
CUMCOUNT_GROUP=[
    ['ip','os'],
    ['ip','app'],
    ['ip','device','os','app'],
    ['ip'],
]



In [5]:
for spec in CUMCOUNT_GROUP:
    day_x = cum_count(spec,day_x)

In [6]:
day_x.dtypes

ip                                                 int32
app                                                int32
device                                             int32
os                                                 int32
channel                                            int32
click_time                                datetime64[ns]
channel_device_mean_target_day9_alpha1           float32
app_os_mean_target_day9_alpha1                   float32
device_mean_target_day9_alpha1                   float32
os_device_mean_target_day9_alpha1                float32
app_mean_target_day9_alpha1                      float32
ip_mean_target_day9_alpha1                       float32
app_device_mean_target_day9_alpha1               float32
channel_mean_target_day9_alpha1                  float32
channel_os_mean_target_day9_alpha1               float32
app_channel_mean_target_day9_alpha1              float32
ip_next_click                                      int32
ip_app_next_click              

In [7]:
day_x= downcast_dtypes(day_x)
day_x.to_feather(PATH/'test_FE.feather')

In [22]:
day_x.tail().T

,18790464,18790465,18790466,18790467,18790468
ip,99442,88046,81398,123236,73516
app,9,23,18,27,12
device,1,1,1,1,2
os,13,37,17,13,27
channel,127,153,265,122,265
click_time,2017-11-10 15:00:00,2017-11-10 15:00:00,2017-11-10 15:00:00,2017-11-10 15:00:00,2017-11-10 15:00:00
channel_device_mean_target_day9_alpha1,0.00102259,0.000144051,0.000136955,0.000426872,3.3205e-08
app_os_mean_target_day9_alpha1,0.00115198,2.13711e-07,0.000442333,0.000571638,2.38668e-05
device_mean_target_day9_alpha1,0.00166388,0.00166388,0.00166388,0.00166388,0.000286772
os_device_mean_target_day9_alpha1,0.0014048,0.00332462,0.00103185,0.0014048,0.000147781


In [9]:
gc.collect()

0

# Group-by aggregation (for each day)

In [10]:
# Define all the groupby transformations
GROUPBY_AGGREGATIONS = [
    
    # Count, for ip-day-hour
    {'groupby': ['ip','day','hour'], 'select': 'channel', 'agg': 'count'},
#     Count, for ip-app
    {'groupby': ['ip', 'app'], 'select': 'channel', 'agg': 'count'},        
    # Count, for ip-app-os
    {'groupby': ['ip', 'app', 'os'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app-day-hour
    {'groupby': ['ip','app','day','hour'], 'select': 'channel', 'agg': 'count'},
    # Mean hour, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'hour', 'agg': 'mean'}, 
    
    # V2 - GroupBy Features #
    #########################
    # Average clicks on app by distinct users; is it an app they return to?
    {'groupby': ['app'], 
     'select': 'ip', 
     'agg': lambda x: float(len(x)) / len(x.unique()), 
     'agg_name': 'AvgViewPerDistinct'
    },
    
    # How popular is the app or channel?
    {'groupby': ['ip'], 'select': 'channel', 'agg': 'count'},
    {'groupby': ['app'], 'select': 'channel', 'agg': 'count'},
    {'groupby': ['channel'], 'select': 'app', 'agg': 'count'},
    
    
    # Size calculation
    {'groupby': ['ip','device','os'], 'select': None, 'agg': 'size'},
    {'groupby': ['ip','device','os','app'], 'select': None, 'agg': 'size'}
    {'groupby': ['ip', 'app', 'os'], 'select': None, 'agg': 'size'}
]

In [3]:
# day_x=get_feather('train_day7_3to16_todo.feather')

In [8]:
# day_x.to_feather(PATH/'train_day7_3to16_todo.feather')

In [11]:
# Apply all the groupby transformations
for spec in GROUPBY_AGGREGATIONS:
    day_x,_=groupby_agg(spec,day_x)

Grouping by ['ip', 'app', 'os'], and aggregating channel with count
Grouping by ['ip', 'app', 'day', 'hour'], and aggregating channel with count
Grouping by ['app'], and aggregating ip with AvgViewPerDistinct
Grouping by ['ip'], and aggregating channel with count
Grouping by ['app'], and aggregating channel with count
Grouping by ['channel'], and aggregating app with count
Grouping by ['ip', 'device', 'os'], and aggregating None with size
Grouping by ['ip', 'device', 'os', 'app'], and aggregating None with size


In [12]:
day_x = downcast_dtypes(day_x)
day_x.to_feather(PATH/'test_FE.feather')

In [33]:
day_x.tail().T

,18790464,18790465,18790466,18790467,18790468
ip,9.944200e+04,8.804600e+04,8.139800e+04,123236.000000,7.351600e+04
app,9.000000e+00,2.300000e+01,1.800000e+01,27.000000,1.200000e+01
device,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,2.000000e+00
os,1.300000e+01,3.700000e+01,1.700000e+01,13.000000,2.700000e+01
channel,1.270000e+02,1.530000e+02,2.650000e+02,122.000000,2.650000e+02
channel_device_mean_target_day9_alpha1,1.022593e-03,1.440514e-04,1.369547e-04,0.000427,3.320501e-08
app_os_mean_target_day9_alpha1,1.151977e-03,2.137107e-07,4.423326e-04,0.000572,2.386684e-05
device_mean_target_day9_alpha1,1.663880e-03,1.663880e-03,1.663880e-03,0.001664,2.867721e-04
os_device_mean_target_day9_alpha1,1.404799e-03,3.324625e-03,1.031854e-03,0.001405,1.477808e-04
app_mean_target_day9_alpha1,9.412590e-04,1.317937e-05,4.727381e-04,0.002019,9.682682e-05


In [14]:
day_x.shape

(18790469, 34)

In [13]:
day_x.columns

Index(['ip', 'app', 'device', 'os', 'channel',
       'channel_device_mean_target_day9_alpha1',
       'app_os_mean_target_day9_alpha1', 'device_mean_target_day9_alpha1',
       'os_device_mean_target_day9_alpha1', 'app_mean_target_day9_alpha1',
       'ip_mean_target_day9_alpha1', 'app_device_mean_target_day9_alpha1',
       'channel_mean_target_day9_alpha1', 'channel_os_mean_target_day9_alpha1',
       'app_channel_mean_target_day9_alpha1', 'ip_next_click',
       'ip_app_next_click', 'ip_channel_next_click', 'ip_os_cumcount',
       'ip_app_cumcount', 'ip_device_os_app_cumcount', 'ip_cumcount', 'day',
       'hour', 'minute', 'second', 'ip_app_os_count_channel',
       'ip_app_day_hour_count_channel', 'app_AvgViewPerDistinct_ip',
       'ip_count_channel', 'app_count_channel', 'channel_count_app',
       'ip_device_os_size_None', 'ip_device_os_app_size_None'],
      dtype='object')

# Time features

In [ ]:
day_x = time_feature(day_x)

In [ ]:
day_x.to_feather('test_FE.feather')

# Mean encoding (frequency) (lagging from previous day)

prev_mean_target. mean_df is saved to merge into validation/test set

TODO: 3to6_prev_mean_target, 7to11_prev_mean_target, 12to16_prev_mean_target